# Markov State Models of Ising model dynamics

Recall that the "true" activation rate at 30˚C (determined from 450 unbiased time traces of the Ising process) is

$\alpha_0 = 1.075 \pm 0.008$ $\text{kHz}$

Here, we will work through the steps of building an MSM from a similar data set of 32 discrete-time trajectories, sampled every 1 µs.  For convenience, we have already simulated kinetic Monte Carlo traces, and converted them to discrete-time trajectories for you.

How close do you think we can get to the true value of $\alpha_0$?


### Step 1:  Load in the trajectory data

The trajectories are stored as $T \times 400$ <code>numpy</code> arrays, where $T$ is the number of snapshots.

In [ ]:
import numpy as np
import glob

### adjust the number of trajectories we want in our dataset by uncommenting
tags = ['a','b','c','d','e','f','g','h']   # 8 trajectories
#tags += ['i','j','k','l','m','n','o','p']  # 16
#tags += ['q','r','s','t','u','v','w','x'] # 24
#tags += ['aa','bb','cc','dd','ee','ff','gg','hh']   # 36

filenames = ['data/%s_all.msmtraj.npy'%s for s in tags]
nfiles = len(filenames)
trajs = [np.load(filenames[trial]) for trial in range(nfiles)]

print 'The shape of a trajectory array:', trajs[0].shape

Let's take a look at the first of the trajectories...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

skip_frames = 1
dq = 1.0/(20*20)   # gating charge/ncells
dt = 1e-6

plt.figure( figsize=(20,4))

plt.subplot(2,1,1)
# Let's make a time trace of the gating charge
timepoints = np.arange(0,trajs[0].shape[0]-skip_frames)*dt  # in seconds
gating_charge = dq*trajs[0][skip_frames:,:].sum(axis=1)
plt.plot(timepoints*1000.0, gating_charge)   # convert to ms
plt.xlim(0,250)
plt.xlabel('time (ms)')


plt.subplot(2,1,2)
# Let's make a histogram of the gating charge
qbins = np.arange(0.0, 1.0+2*dq, 2*dq)
qcenters = (qbins[0:-1]+qbins[1:])/2.0
counts, edges = np.histogram(gating_charge, bins=qbins)
plt.plot(qcenters,counts)
plt.xlabel('gating charge (e)')



### Step 2:  Perform time-lagged Indepedent Component Analysis (tICA)

Next, the tICA algorithm is used perform dimensionality reduction to a subspace of tICs representing the degrees of freedom along which the most time-correlated (i.e. slowest) motions occur.

This step requires constructing and diagonalizing a 400 $\times$ 400 correlation matrix, so it may take some time.

The output is:

(1) A series of trajectories in the new coordinate system, i.e. the values of each tIC over time.  These trajectories are stored in <code>tica_coords</code>, and saved to file.

(2) Instantiation and training of <code>tica</code>, an <code>sklearn</code>-style model object.  This object contains all the training data and model parameters, including the tICA components.

In [ ]:
from msmbuilder.decomposition import tICA, PCA

# Choose the number of tICA components used to project the data.
ntica = 10

print 'Computing tICA decomposition for ntica = %d ...'%ntica
tica = tICA(n_components=ntica, lag_time=1)
tica_coords = tica.fit_transform(trajs)

for i in range(len(tica_coords)):
    tag = tags[i]
    filename = 'data/tica/%s_all.tica%d.npy'%(tag,ntica)
    np.save(filename,tica_coords[i])
    print '\tWrote', filename

Let's visualize the first ten tICA components...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure( figsize=(20,8))

npanels = 10
for j in range(min(npanels,ntica)):
    plt.subplot(2,5,j+1)
    #print 'tica.eigenvectors_[:,j].min()', tica.eigenvectors_[:,j].min()
    #print 'tica.eigenvectors_[:,j].max()', tica.eigenvectors_[:,j].max()
    plt.pcolor( tica.eigenvectors_[:,j].reshape( (20,20) ), cmap='RdBu', vmin=-0.15, vmax=0.15)
    if j == 0:
        plt.colorbar()
    plt.title('tICA $\\vec{e}_%d$'%(j+1))


Let's also visualize (some of) the projection of the trajectory data onto tIC$_1$ and tIC$_2$, the first two tICA components....

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


plt.figure( figsize=(16,8))
skip_points = 1   #  adjust this if you want to subsample the data

number_to_show = 2 #  adjust the number of trajectories to visualize
for i in range(len(tica_coords[0:(number_to_show-1)])):  
    plt.plot(tica_coords[i][::skip_points,0],tica_coords[i][::skip_points,1], linewidth=0.1)
    plt.xlabel('tIC$_1$', fontsize=24)
    plt.ylabel('tIC$_2$', fontsize=24)


### Step 3: Clustering and visualization
See: http://msmbuilder.org/3.7.0/examples/Fs-Peptide-in-RAM.html

First, let's generate some initial points to start the *kmeans* clustering. This is important because random selection of cluster generators based on the samples alone will be biased toward the two basins.

In [ ]:
# find the max and min of the tica data and scale the initial points accordingly
tica_mins = np.zeros( (len(tica_coords),ntica) )
tica_maxs = np.zeros( (len(tica_coords),ntica) ) 
for i in range(len(tica_coords)):
    tica_mins[i,:] = np.min(tica_coords[i], axis=0)
    tica_maxs[i,:] = np.max(tica_coords[i], axis=0)
best_tica_mins = np.min(tica_mins, axis=0)
best_tica_maxs = np.min(tica_maxs, axis=0)
print 'best_tica_mins', best_tica_mins
print 'best_tica_maxs', best_tica_maxs


#######  Adjust this parameter to change the number of clusters.  
#######  WARNING: Large numbers of clusters and large data sets will consume time and memory!
number_of_clusters = 100 # 2000

# NOTE: ntica is defined in the cells above
initial_generators = np.random.random( (number_of_clusters,ntica) )
widths = best_tica_maxs - best_tica_mins
for i in range(ntica):
    initial_generators[:,i] = initial_generators[:,i]*widths[i] + best_tica_mins[i]

Clustering proceeds in two steps: (1) using a subset of the data to find the cluster generators, and (2) assigning the remaining data to the generators.  This saves time and memory, with (hopefully) minimally loss of accuracy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from msmbuilder.cluster import MiniBatchKMeans, KMeans
#clusterer = MiniBatchKMeans(n_clusters=number_of_clusters, init=initial_generators)
clusterer = KMeans(n_clusters=number_of_clusters, init=initial_generators)


########################################################
### 2) cluster using 1/100 of the data
subset_stride = 100  #  Adjustable parameter

# create a subset of the data to do initial cluster on
tica_coords_subset = [t[0::subset_stride,:] for t in tica_coords]
clusterer.fit(tica_coords_subset)

# next, assign the rest of the data
clustered_trajs = clusterer.predict(tica_coords)
# print(clusterer.summarize())


Let's visualize where the generators lie on the tIC1 tIC2 landscape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# join list of tICA trajectories into a single traj for plotting
txx = np.concatenate(tica_coords)
plt.hexbin(txx[:,0], txx[:,1], bins='log', mincnt=1, cmap='viridis')
plt.scatter(clusterer.cluster_centers_[:,0],
            clusterer.cluster_centers_[:,1], 
            s=100, c='w')

We can also visualize the higher-dimensions using tICA cross-plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure( figsize=(16,16))
npanels = 6

for i in range(npanels):
    for j in range(npanels):
        if i < j:
            plt.subplot(npanels,npanels, i+j*npanels + 1)
            txx = np.concatenate(tica_coords)
            plt.hexbin(txx[0::20,i], txx[0::20,j], bins='log', mincnt=1, cmap='viridis')
            plt.scatter(clusterer.cluster_centers_[:,i], clusterer.cluster_centers_[:,j], s=50, c='w')
            if j == npanels-1:
                plt.xlabel('tIC$_%d$'%i, fontsize=16)
            if i == 0:
                plt.ylabel('tIC$_%d$'%j, fontsize=16)

### Step 4: Calculation of implied timescales

Finally, we calculate implied timescales with respect to lag time to find slowest rate process

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from msmbuilder.msm import ContinuousTimeMSM, MarkovStateModel, implied_timescales

lag_times = np.array([1, 2, 4, 10, 15, 20, 30, 50, 75, 100, 200])  # in units of steps
n_timescales = 10
dt = 1.0e-6

msm_timescales = implied_timescales(clustered_trajs, lag_times, n_timescales=n_timescales, msm=MarkovStateModel(verbose=False))
for i in range(n_timescales):
   plt.plot(lag_times*dt*1.0e6, msm_timescales[:, i]*dt*1.0e3, 'o-')  # lagtime in us,  implied timescales in ms

plt.title('Discrete-time MSM Relaxation Timescales')
plt.semilogy()
plt.xlabel('lag time ($\\mu s$)')
plt.ylabel('implied timescales (ms)')

print '### spectral estimates ###'
#lag_times = [1, 2, 4, 10, 15, 20, 30, 50, 75, 100, 200]
tau = msm_timescales[list(lag_times).index(100), 0]
print 'implied timescale (using 100 us lagtime):', tau*dt*1.0e3, 'ms'
# Make a star on the plot indicating tau at 100 µs lagtime
plt.plot(100,tau*dt*1.0e3,'y*', markersize=16)
print 'average dwell time (ms), 1/k = 2*tau', 2*tau*dt*1.0e3
print 'Spectral estimate of crossing rate,  k = (1/tau)/2:', 0.5/(tau*dt*1.0e3), 'kHz'


## Questions

How do the estimated rates change with
* the number of states in the MSM?
* the number of trajectories using to train the MSM?
* the number of tICA components used in the projection?
* the clustering method?

### Further reading

The above questions have to do with the problem of *model selection*.  Fortunately, there is a rigorous **variational principle** we can invoke to select optimal "hyperparameters" such as the number of states.


**A variational approach to model selection.** Christof Schütte (ZIB) and Frank Noé (FUB) groups have shown that discretation of the conformational space leads to errors in approximating the eigenvectors of the (continuous) dynamical transfer operator, and the so-called Variational Approach to Conformational Dynamics (VAC), which says that eny error in the eigenvectors will always *underestimate* the implied timescales.  Therefore, as long as we can control for finite sampling and/or overfitting errors, the model with the *slowest* timescales is the optimal model.

**Model selection in face of sampling errors**.  Using such a variational principle in practice can be difficult because of the presence of sampling noise.  McGibbon and Pande have a very nice method for dealing with this, by calculating the so-called Generalized matrix Raleigh quotient (GMRQ).  In order to avoid over-fitting to a finite sample of data, a cross-validation procedure can be performed in which the data is partioned into a number of separate training and testing trials.

#### References

Nüske, F., Keller, B. G., Perez-Hernandez, G., Mey, A. S. J. S., & Noé, F. (2014). Variational Approach to Molecular Kinetics. Journal of Chemical Theory and Computation, 10(4), 1739–1752. http://doi.org/10.1021/ct4009156

Prinz, J.-H., Wu, H., Sarich, M., Keller, B., Senne, M., Held, M., et al. (2011). Markov models of molecular kinetics: generation and validation. The Journal of Chemical Physics, 134(17), 174105. http://doi.org/10.1063/1.3565032

McGibbon, R. T., & Pande, V. S. (2015). Variational cross-validation of slow dynamical modes in molecular kinetics. The Journal of Chemical Physics, 142(12), 124105. http://doi.org/10.1063/1.4916292


